In [1]:
# import libraries

import csv
import requests
import time
import datetime
import pandas as pd
from bs4 import BeautifulSoup

# for email functionality
import smtplib

In [2]:
# Connect to the first target Website URL and pull in data

URL1 = 'https://www.amazon.co.uk/Nintendo-Switch-Pokemon-Violet-Pok%C3%A9mon-Violet-Switch/dp/B0B2X4BMV2/ref=sr_1_1?crid=7ILLBVLK4WC&keywords=pokemon+violet&qid=1679752151&sprefix=pokemon+vio%2Caps%2C162&sr=8-1'
# user agent found from https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page1 = requests.get(URL1, headers=headers) # request data from specified URL

page1_code = BeautifulSoup(page1.content, "html.parser") # pulling in html content from page

page1_code = BeautifulSoup(page1_code.prettify()) # prettify formatting of page_code html data

title = page1_code.find(id="productTitle").get_text() # find the product title by searching for the element id
price1_pounds = page1_code.find("span", {"class": "a-price-whole"}).get_text() # price is split into pounds and pence - find the price by searching for the class as no id present
price1_pence = page1_code.find("span", {"class": "a-price-fraction"}).get_text()


In [3]:
# Connect to the second target Website URL and pull in data

URL2 = 'https://www.game.co.uk/en/pok-mon-violet-2878152'
 
page2 = requests.get(URL2, headers=headers) # request data from specified URL

page2_code = BeautifulSoup(page2.content, "html.parser") # pulling in html content from page

page2_code = BeautifulSoup(page2_code.prettify()) # prettify formatting of page_code html data
 
price2 = page2_code.find("strong", {"class": "btnPrice"}).get_text()  # find the product sell price by searching for the element class
 

In [4]:
# tidy data removing enpty space to be ready to import to csv file
title = title.strip()
price1_pounds = price1_pounds.strip()
price1_pounds = price1_pounds.replace(" ", "") # price_pounds has more white space to remove because of a nested element
price1_pounds = price1_pounds.replace("\n", "")
price1_pence = price1_pence.strip()
total_price1 = price1_pounds + price1_pence # concatenate price

price2 = price2.strip()
price2 = price2.replace("£", "")

date_of_scrape = datetime.date.today() # date scrape occoured

In [5]:
# calculate price difference and which retailer is cheapest
price_difference = float(total_price1) - float(price2) # calculate price difference

cheapest_retailer = "" # variable declaration to store name of cheapest retailer 
if price_difference < 0: # find out which retailer is cheaper
    cheapest_retailer = "Amazon"
    price_difference = float(price2) - float(total_price1) # if Game is the cheapest retailer redefine prcie_difference so the value is positive
elif price_difference > 0:
    cheapest_retailer = "Gane"
else:
    cheapest_retailer = "Same price" 

price_difference = round(price_difference, 2) # Round the price difference to two decimal places


In [6]:
# declare variables for csv file data
header = ["Title", "Price at Amazon", "Price at Game", "Price difference", "Cheapest Retailer", "Date of Scrape"] # column headings 
data = [title, total_price1, price2, price_difference, cheapest_retailer, date_of_scrape] # row data
""" 
# create csv file - this only needs to be run once, to create the initial file, further addition to file is handled later so commented out so as not to accidentally overwrite additional file data
with open("PriceComparisonDataset.csv", "w", newline="", encoding="UTF8") as f:
    writer = csv.writer(f)
    writer.writerow(header) # write column headdings in first row
    writer.writerow(data) # write data in next row below headings
 """

' \n# create csv file - this only needs to be run once, to create the initial file, further addition to file is handled later so commented out so as not to accidentally overwrite additional file data\nwith open("PriceComparisonDataset.csv", "w", newline="", encoding="UTF8") as f:\n    writer = csv.writer(f)\n    writer.writerow(header) # write column headdings in first row\n    writer.writerow(data) # write data in next row below headings\n '

In [7]:
data_frame = pd.read_csv(r"C:\Users\Simeon\Documents\Data Science\Price comparison web scrape\PriceComparisonDataset.csv")

print(data_frame)


                              Title  Price at Amazon  Price at Game  \
0  Pokémon Violet (Nintendo Switch)            39.64          39.99   
1  Pokémon Violet (Nintendo Switch)            39.64          39.99   
2  Pokémon Violet (Nintendo Switch)            39.64          39.99   
3  Pokémon Violet (Nintendo Switch)            39.64          39.99   
4  Pokémon Violet (Nintendo Switch)            39.64          39.99   
5  Pokémon Violet (Nintendo Switch)            39.64          39.99   

   Price difference Cheapest Retailer Date of Scrape  
0              0.35            Amazon     2023-03-25  
1              0.35            Amazon     2023-03-25  
2              0.35            Amazon     2023-03-25  
3              0.35            Amazon     2023-03-25  
4              0.35            Amazon     2023-03-25  
5              0.35            Amazon     2023-03-25  


In [8]:
# combine code into a function which we can automate

def price_comparison():
    # Connect to the first target Website URL and pull in data
    URL1 = 'https://www.amazon.co.uk/Nintendo-Switch-Pokemon-Violet-Pok%C3%A9mon-Violet-Switch/dp/B0B2X4BMV2/ref=sr_1_1?crid=7ILLBVLK4WC&keywords=pokemon+violet&qid=1679752151&sprefix=pokemon+vio%2Caps%2C162&sr=8-1'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page1 = requests.get(URL1, headers=headers) # request data from specified URL
    page1_code = BeautifulSoup(page1.content, "html.parser") # pulling in html content from page
    page1_code = BeautifulSoup(page1_code.prettify()) # prettify formatting of page_code html data

    title = page1_code.find(id="productTitle").get_text() # find the product title by searching for the element id
    price1_pounds = page1_code.find("span", {"class": "a-price-whole"}).get_text() # price is split into pounds and pence - find the price by searching for the class as no id present
    price1_pence = page1_code.find("span", {"class": "a-price-fraction"}).get_text()

    # Connect to the second target Website URL and pull in data

    URL2 = 'https://www.game.co.uk/en/pok-mon-violet-2878152'

    page2 = requests.get(URL2, headers=headers) # request data from specified URL
    page2_code = BeautifulSoup(page2.content, "html.parser") # pulling in html content from page
    page2_code = BeautifulSoup(page2_code.prettify()) # prettify formatting of page_code html data

    price2 = page2_code.find("strong", {"class": "btnPrice"}).get_text()  # find the product sell price by searching for the element class


    # tidy data removing enpty space to be ready to import to csv file
    title = title.strip()
    price1_pounds = price1_pounds.strip()
    price1_pounds = price1_pounds.replace(" ", "") # price_pounds has more white space to remove because of a nested element
    price1_pounds = price1_pounds.replace("\n", "")
    price1_pence = price1_pence.strip()
    total_price1 = price1_pounds + price1_pence # concatenate price

    price2 = price2.strip()
    price2 = price2.replace("£", "")

    date_of_scrape = datetime.date.today() # date scrape occoured
    
    # calculate price difference and which retailer is cheapest
    price_difference = float(total_price1) - float(price2) # calculate price difference
    cheapest_retailer = "" # variable declaration to store name of cheapest retailer 

    if price_difference < 0: # find out which retailer is cheaper
        cheapest_retailer = "Amazon"
        price_difference = float(price2) - float(total_price1) # if Game is the cheapest retailer redefine prcie_difference so the value is positive
    elif price_difference > 0:
        cheapest_retailer = "Gane"
    else:
        cheapest_retailer = "Same price" 

    price_difference = round(price_difference, 2) # Round the price difference to two decimal places

    # declare variables for csv file data
    header = ["Title", "Price at Amazon", "Price at Game", "Price difference", "Cheapest Retailer", "Date of Scrape"] # column headings 
    data = [title, total_price1, price2, price_difference, cheapest_retailer, date_of_scrape] # row data
    
    # append data to the csv to track price changes over time
    with open("PriceComparisonDataset.csv", "a+", newline="", encoding="UTF8") as f:  # write data in the next avaliable row - appends to new row "a+" argument
        writer = csv.writer(f)
        writer.writerow(data) 

In [ ]:
# automate price_comparison to run once per day
while(True):
    price_comparison()
    time.sleep(86400)